In [1]:
import os
os.environ["mode"] = 'dev'

In [2]:

import sys
sys.path.append(r'C:\Users\Mi\Desktop\code\git\python-utils-ak')
sys.path.append(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat')
from utils_ak.interactive_imports import *

In [3]:


def _import_globals(module):
    for x in dir(module):
        if not x.startswith('_'):
            globals()[x] = getattr(module, x)
            
def _load(module_obj, module_name=None, reload=False, import_globals=False):
    module = load(module_obj, module_name, reload, import_globals=False)
    if import_globals:
        _import_globals(module)
    return module

def load_sources():
    _load(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\utils/time.py', import_globals=True, reload=True)
    _load(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\utils/block.py', import_globals=True, reload=True)
    _load(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\utils/color.py', import_globals=True, reload=True)
    _load(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\utils/drawing.py', import_globals=True, reload=True)
    _load(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\utils/interval.py', import_globals=True, reload=True)
    
    _load(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\blocks.py', import_globals=True, reload=True)

    _load(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\models.py', import_globals=True, reload=True)
    _load(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\algo.py', import_globals=True, reload=True)
    _load(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\style.py', import_globals=True, reload=True)
    _load(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\validation.py', import_globals=True, reload=True)
    _load(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\utils\openpyxl_wrapper.py', import_globals=True, reload=True)
#     _load(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\utils\excel_client.py', import_globals=True, reload=True)

load_sources()

In [4]:
remove('output.xlsx')
load_sources()
request = anyconfig.load(r"C:\Users\Mi\Desktop\umalat\schedules\2020.12.04.json")
date = cast_datetime(request['Date'])
df = gen_request_df(request)
root = make_schedule(request, date)
wb = draw_workbook(root, mode='prod')
wb.save('_drawing.xlsx')
rename('_drawing.xlsx', 'output.xlsx')
# open_file_in_os('output.xlsx')

Starting from 24 salt 48 24
